In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf
import cv2
import os.path
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


CNN

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
dataTrain = Path("../content/gdrive/My Drive/Colab Notebooks/dataset/training_set")
filepath = list(dataTrain.glob(r"**/*.jpg"))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))

In [ ]:
filepath = pd.Series(filepath, name='Filepath').astype(str)
labels = pd.Series(labels,name="Labels")

In [ ]:
# Read the images and store it to ListA
ListA = []
for i in filepath:
    image = cv2.imread(i,0)
    ListA.append(np.array(image,"uint8"))

In [ ]:
ListA = pd.Series(ListA,name="Array") # Vectorized image
MainData = pd.concat([ListA,labels],axis=1) # Construct main data and copy it
data = MainData.copy() # to protect main data
print(data["Labels"].value_counts())

In [ ]:
figure = plt.figure(figsize=(15,8))
plt.imshow(data["Array"][10])

In [ ]:
print(data.head())

In [ ]:
data['Array'].iloc[0].shape

In [ ]:
SecondData = pd.concat([filepath,labels],axis=1)
x = SecondData["Filepath"]
y = SecondData["Labels"]

# Encode categorized variables, similar to Getdummy
encode = LabelEncoder() 
x = encode.fit_transform(x)
y = encode.fit_transform(y)
SecondData = pd.concat([filepath,labels],axis=1)

# scale Xs
x = x/max(x)

# convert type to float and reshape x and y
x = np.asarray(x).astype("float32")
x = x.reshape(len(x),1,1)
y = np.asarray(y).astype("float32")
y = y.reshape((len(y)),1,1)


In [ ]:
from google.colab.patches import cv2_imshow
path = SecondData.iloc[0][0]
image = cv2.imread(path)
cv2_imshow(image)

In [ ]:
x = SecondData["Filepath"]
y = SecondData["Labels"]
encode = LabelEncoder()
x = encode.fit_transform(x)
y = encode.fit_transform(y)

In [ ]:
x

In [ ]:
y

In [ ]:
x = np.asarray(x).astype("float32")
x = x.reshape(126,1,1)

y = np.asarray(y).astype("float32")
y = y.reshape(126,1,1)

In [ ]:
# xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.1,random_state=42)
# xTrain

## ANN Implementation

## CNN Implementation

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.layers.core import Dropout
from keras.metrics import categorical_crossentropy
from keras.callbacks import ModelCheckpoint

In [ ]:
dataTrain = Path("../content/gdrive/My Drive/Colab Notebooks/dataset/training_set")
dataValidation = Path("../content/gdrive/My Drive/Colab Notebooks/dataset/validation_set")
# dataTest = Path("../content/gdrive/My Drive/Colab Notebooks/dataset/test_set")
filepath = list(dataTrain.glob(r"**/*.jpg"))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))

selectedClasses = os.listdir(dataTrain)

In [ ]:
len(selectedClasses)

In [ ]:
train_datagen = ImageDataGenerator(#featurewise_center = True,
                              rescale = 1./255,
                              rotation_range = 20,
                              fill_mode = 'nearest',
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True,)
                                        
train_generator = train_datagen.flow_from_directory(
        dataTrain,
        target_size=(224, 224), 
        batch_size= 16,
        #classes = selectedClasses,
        shuffle=True,
        class_mode='categorical')

In [ ]:
val_generator = train_datagen.flow_from_directory(
        dataValidation,
        target_size=(224, 224), 
        batch_size= 16,
        #classes = selectedClasses,
        shuffle=True,
        class_mode='categorical')

validation_steps= val_generator.samples//64
print(validation_steps)

In [ ]:
base_model = InceptionResNetV2(weights='imagenet',
                         include_top=False,
                        input_shape = (224,224,3))
base_model.trainable = False
x = base_model.output
x = keras.layers.GlobalMaxPooling2D()(x)
x = BatchNormalization()(x)
x = Dense(50,activation='relu')(x)
predictions = Dense(len(selectedClasses), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
# model.summary()

model.compile(optimizer = Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
filepath = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
model_checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

train_generator.reset()
val_generator.reset()

historyTM = model.fit(train_generator,
                    validation_data=val_generator,
                    steps_per_epoch = 9,
                    validation_steps = 1,
                    epochs = 30, 
                    verbose = 1
                    callbacks = [model_checkpoint]
                   ) 



In [ ]:
# Plot the learning curves
acc = historyTM.history['accuracy']
val_acc = historyTM.history['val_accuracy']

loss = historyTM.history['loss']
val_loss = historyTM.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='lower right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# # test_generator = train_datagen.flow_from_directory(
# #         dataTest,
# #         target_size=(224, 224), 
# #         batch_size= 16,
# #         #classes = selectedClasses,
# #         shuffle=True,
# #         class_mode='categorical')
# filepath = list(dataTest.glob(r"**/*.jpg"))
# labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))
# for i in filepath:
#   print(i)
#   #predictions = np.argmax( model.predict(i),axis=1)
#   model.predict(i)


In [ ]:
# base_model.trainable = True
# layers_to_freeze = 50
# for i in base_model.layers[:layers_to_freeze]:
#     i.trainable = False


# model.compile(optimizer = Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
# train_datagen.fit(train_generator)
# fine_tunemodel = model.fit(train_generator,
#                           epochs = 30,
#                           initial_epoch = historyTM.epoch[-1],
#                           validation_data = val_generator,
#                           verbose = 1,
#                           steps_per_epoch = 9,
#                           validation_steps = 1)

# model.evaluate(val_generator)

In [ ]:
base_model.trainable = True
layers_to_freeze = 50
for i in base_model.layers[:layers_to_freeze]:
    i.trainable = False
    
model.compile(optimizer = Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fine_tunemodel = model.fit(train_generator,
                          epochs = 50,
                          initial_epoch = 0,
                          validation_data = val_generator,
                          verbose = 1,
                          steps_per_epoch = 9,
                          validation_steps = 1)